In [14]:
import pickle
import pandas as pd
import numpy as np
from pyspark.sql import DataFrame
from aeon.classification.distance_based import ProximityTree, ProximityForest
import logging

from pyspark.sql import SparkSession
import os
from pyspark.sql import SparkSession
from data_ingestion import DataIngestion
from preprocessing import Preprocessor
from prediction_manager import PredictionManager
from local_model_manager import LocalModelManager
from evaluation import Evaluator
from utilities import show_compact
import time
import json
from random import sample
from dtaidistance import dtw

## ---


In [15]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("GenericRDD").getOrCreate()

# Access the SparkContext
sc = spark.sparkContext

# ---

In [16]:
data = [
    {"label": 1, "time_series": [1.0, 2.1, 3.2, 4.3, 5.4]},
    {"label": 2, "time_series": [2.0, 3.1, 4.2, 5.3, 6.4]},
    {"label": 3, "time_series": [3.0, 4.1, 5.2, 6.3, 7.4]},
    {"label": 4, "time_series": [4.0, 5.1, 6.2, 7.3, 8.4]},
    {"label": 1, "time_series": [1.5, 2.6, 3.7, 4.8, 5.9]},
    {"label": 2, "time_series": [2.5, 3.6, 4.7, 5.8, 6.9]},
    {"label": 3, "time_series": [3.5, 4.6, 5.7, 6.8, 7.9]},
    {"label": 4, "time_series": [4.5, 5.6, 6.7, 7.8, 8.9]}
]

rdd = sc.parallelize(data)

In [17]:
rdd = rdd.repartition(2)
rdd.getNumPartitions()

2

# adding exemplar column

In [18]:
def sample_and_add_column(iterator):
    partition_data = list(iterator)
    sampled_element = sample(partition_data, 1)[0]['time_series']
    return iter([{**row, "exemplar": sampled_element} for row in partition_data])

rdd_with_sampled_column = rdd.mapPartitions(sample_and_add_column)

# Collect and print the updated RDD
for row in rdd_with_sampled_column.collect():
    print(row)

{'label': 3, 'time_series': [3.0, 4.1, 5.2, 6.3, 7.4], 'exemplar': [1.5, 2.6, 3.7, 4.8, 5.9]}
{'label': 4, 'time_series': [4.0, 5.1, 6.2, 7.3, 8.4], 'exemplar': [1.5, 2.6, 3.7, 4.8, 5.9]}
{'label': 1, 'time_series': [1.5, 2.6, 3.7, 4.8, 5.9], 'exemplar': [1.5, 2.6, 3.7, 4.8, 5.9]}
{'label': 2, 'time_series': [2.5, 3.6, 4.7, 5.8, 6.9], 'exemplar': [1.5, 2.6, 3.7, 4.8, 5.9]}
{'label': 3, 'time_series': [3.5, 4.6, 5.7, 6.8, 7.9], 'exemplar': [1.5, 2.6, 3.7, 4.8, 5.9]}
{'label': 1, 'time_series': [1.0, 2.1, 3.2, 4.3, 5.4], 'exemplar': [2.0, 3.1, 4.2, 5.3, 6.4]}
{'label': 2, 'time_series': [2.0, 3.1, 4.2, 5.3, 6.4], 'exemplar': [2.0, 3.1, 4.2, 5.3, 6.4]}
{'label': 4, 'time_series': [4.5, 5.6, 6.7, 7.8, 8.9], 'exemplar': [2.0, 3.1, 4.2, 5.3, 6.4]}


# calculating DTW distance using time series and exemplar columns

In [19]:
# def calc_dtw_distance(iterator):
#     partition_data = list(iterator)
#     time_series = partition_data['time_series']
#     exemplar = partition_data['exemplar']
#     dtw_distance = dtw.distance(time_series, exemplar)
#     return iter([{**row, "dtw_distance": dtw_distance} for row in partition_data])

def calc_dtw_distance(iterator):
    partition_data = list(iterator)
    updated_rows = []
    
    for row in partition_data:
        time_series = row['time_series']
        exemplar = row['exemplar']
        
        dtw_distance = dtw.distance(time_series, exemplar)
        
        updated_row = {**row, "dtw_distance": dtw_distance}
        updated_rows.append(updated_row)
    return iter(updated_rows)

rdd_with_dtw = rdd_with_sampled_column.mapPartitions(calc_dtw_distance)
for row in rdd_with_dtw.collect():
    print(row)

{'label': 3, 'time_series': [3.0, 4.1, 5.2, 6.3, 7.4], 'exemplar': [1.5, 2.6, 3.7, 4.8, 5.9], 'dtw_distance': 2.2671568097509267}
{'label': 4, 'time_series': [4.0, 5.1, 6.2, 7.3, 8.4], 'exemplar': [1.5, 2.6, 3.7, 4.8, 5.9], 'dtw_distance': 4.085339643163099}
{'label': 1, 'time_series': [1.5, 2.6, 3.7, 4.8, 5.9], 'exemplar': [1.5, 2.6, 3.7, 4.8, 5.9], 'dtw_distance': 0.0}
{'label': 2, 'time_series': [2.5, 3.6, 4.7, 5.8, 6.9], 'exemplar': [1.5, 2.6, 3.7, 4.8, 5.9], 'dtw_distance': 1.42828568570857}
{'label': 3, 'time_series': [3.5, 4.6, 5.7, 6.8, 7.9], 'exemplar': [1.5, 2.6, 3.7, 4.8, 5.9], 'dtw_distance': 3.1208973068654466}
{'label': 1, 'time_series': [1.0, 2.1, 3.2, 4.3, 5.4], 'exemplar': [2.0, 3.1, 4.2, 5.3, 6.4], 'dtw_distance': 1.42828568570857}
{'label': 2, 'time_series': [2.0, 3.1, 4.2, 5.3, 6.4], 'exemplar': [2.0, 3.1, 4.2, 5.3, 6.4], 'dtw_distance': 0.0}
{'label': 4, 'time_series': [4.5, 5.6, 6.7, 7.8, 8.9], 'exemplar': [2.0, 3.1, 4.2, 5.3, 6.4], 'dtw_distance': 4.0853396431630

# WORKS FOR ANY NUM OF PARTITIONS AND EXEMPLARS

In [20]:
def create_sample_and_add_column_function(num_exemplars):
    def sample_and_add_column(iterator):
        partition_data = list(iterator)
        exemplars = []
        for row in sample(partition_data, min(num_exemplars, len(partition_data))):
            exemplars.append(row['time_series'])
        return iter([{**row, "exemplars": exemplars} for row in partition_data])
    return sample_and_add_column

# Example usage
num_exemplars = 2
sample_and_add_column = create_sample_and_add_column_function(num_exemplars)
rdd_with_exemplar_column = rdd.mapPartitions(sample_and_add_column)
for row in rdd_with_exemplar_column.collect():
    print(row)

print(f'\nrdd num partitions: {rdd_with_exemplar_column.getNumPartitions()}')

{'label': 3, 'time_series': [3.0, 4.1, 5.2, 6.3, 7.4], 'exemplars': [[1.5, 2.6, 3.7, 4.8, 5.9], [2.5, 3.6, 4.7, 5.8, 6.9]]}
{'label': 4, 'time_series': [4.0, 5.1, 6.2, 7.3, 8.4], 'exemplars': [[1.5, 2.6, 3.7, 4.8, 5.9], [2.5, 3.6, 4.7, 5.8, 6.9]]}
{'label': 1, 'time_series': [1.5, 2.6, 3.7, 4.8, 5.9], 'exemplars': [[1.5, 2.6, 3.7, 4.8, 5.9], [2.5, 3.6, 4.7, 5.8, 6.9]]}
{'label': 2, 'time_series': [2.5, 3.6, 4.7, 5.8, 6.9], 'exemplars': [[1.5, 2.6, 3.7, 4.8, 5.9], [2.5, 3.6, 4.7, 5.8, 6.9]]}
{'label': 3, 'time_series': [3.5, 4.6, 5.7, 6.8, 7.9], 'exemplars': [[1.5, 2.6, 3.7, 4.8, 5.9], [2.5, 3.6, 4.7, 5.8, 6.9]]}
{'label': 1, 'time_series': [1.0, 2.1, 3.2, 4.3, 5.4], 'exemplars': [[2.0, 3.1, 4.2, 5.3, 6.4], [4.5, 5.6, 6.7, 7.8, 8.9]]}
{'label': 2, 'time_series': [2.0, 3.1, 4.2, 5.3, 6.4], 'exemplars': [[2.0, 3.1, 4.2, 5.3, 6.4], [4.5, 5.6, 6.7, 7.8, 8.9]]}
{'label': 4, 'time_series': [4.5, 5.6, 6.7, 7.8, 8.9], 'exemplars': [[2.0, 3.1, 4.2, 5.3, 6.4], [4.5, 5.6, 6.7, 7.8, 8.9]]}

rdd num

In [21]:
def calc_dtw_distance(iterator):
    partition_data = list(iterator)
    updated_rows = []
    
    for row in partition_data:
        time_series = row['time_series']
        exemplars = row['exemplars']
        
        # Calculate DTW distances for each exemplar
        dtw_distances = [dtw.distance(time_series, exemplar) for exemplar in exemplars]
        
        # Add each DTW distance as a separate column
        updated_row = {**row}
        for i, dtw_distance in enumerate(dtw_distances):
            updated_row[f"dtw_distance_exemplar_{i+1}"] = dtw_distance
        
        updated_rows.append(updated_row)
    
    return iter(updated_rows)

# Example usage
rdd_with_dtw = rdd_with_exemplar_column.mapPartitions(calc_dtw_distance)
for row in rdd_with_dtw.collect():
    print(row)

print(f'\nrdd num partitions: {rdd_with_dtw.getNumPartitions()}')

{'label': 3, 'time_series': [3.0, 4.1, 5.2, 6.3, 7.4], 'exemplars': [[1.5, 2.6, 3.7, 4.8, 5.9], [2.5, 3.6, 4.7, 5.8, 6.9]], 'dtw_distance_exemplar_1': 2.2671568097509267, 'dtw_distance_exemplar_2': 1.1180339887498947}
{'label': 4, 'time_series': [4.0, 5.1, 6.2, 7.3, 8.4], 'exemplars': [[1.5, 2.6, 3.7, 4.8, 5.9], [2.5, 3.6, 4.7, 5.8, 6.9]], 'dtw_distance_exemplar_1': 4.085339643163099, 'dtw_distance_exemplar_2': 2.2671568097509267}
{'label': 1, 'time_series': [1.5, 2.6, 3.7, 4.8, 5.9], 'exemplars': [[1.5, 2.6, 3.7, 4.8, 5.9], [2.5, 3.6, 4.7, 5.8, 6.9]], 'dtw_distance_exemplar_1': 0.0, 'dtw_distance_exemplar_2': 1.42828568570857}
{'label': 2, 'time_series': [2.5, 3.6, 4.7, 5.8, 6.9], 'exemplars': [[1.5, 2.6, 3.7, 4.8, 5.9], [2.5, 3.6, 4.7, 5.8, 6.9]], 'dtw_distance_exemplar_1': 1.42828568570857, 'dtw_distance_exemplar_2': 0.0}
{'label': 3, 'time_series': [3.5, 4.6, 5.7, 6.8, 7.9], 'exemplars': [[1.5, 2.6, 3.7, 4.8, 5.9], [2.5, 3.6, 4.7, 5.8, 6.9]], 'dtw_distance_exemplar_1': 3.1208973068

In [22]:
def assign_closest_exemplar(iterator):
    partition_data = list(iterator)

    for row in partition_data:
        # Check if there are DTW distances for exemplars
        exemplar_distances = {key: value for key, value in row.items() if key.startswith("dtw_distance_exemplar_")}
        
        if exemplar_distances:
            # Find the exemplar with the smallest DTW distance
            closest_exemplar = min(exemplar_distances, key=exemplar_distances.get)
            
            # Assign the closest exemplar to the row
            row["closest exemplar"] = closest_exemplar

    return iter(partition_data)

# Example usage
rdd_with_classification = rdd_with_dtw.mapPartitions(assign_closest_exemplar)
for row in rdd_with_classification.collect():
    print(row)

{'label': 3, 'time_series': [3.0, 4.1, 5.2, 6.3, 7.4], 'exemplars': [[1.5, 2.6, 3.7, 4.8, 5.9], [2.5, 3.6, 4.7, 5.8, 6.9]], 'dtw_distance_exemplar_1': 2.2671568097509267, 'dtw_distance_exemplar_2': 1.1180339887498947, 'closest exemplar': 'dtw_distance_exemplar_2'}
{'label': 4, 'time_series': [4.0, 5.1, 6.2, 7.3, 8.4], 'exemplars': [[1.5, 2.6, 3.7, 4.8, 5.9], [2.5, 3.6, 4.7, 5.8, 6.9]], 'dtw_distance_exemplar_1': 4.085339643163099, 'dtw_distance_exemplar_2': 2.2671568097509267, 'closest exemplar': 'dtw_distance_exemplar_2'}
{'label': 1, 'time_series': [1.5, 2.6, 3.7, 4.8, 5.9], 'exemplars': [[1.5, 2.6, 3.7, 4.8, 5.9], [2.5, 3.6, 4.7, 5.8, 6.9]], 'dtw_distance_exemplar_1': 0.0, 'dtw_distance_exemplar_2': 1.42828568570857, 'closest exemplar': 'dtw_distance_exemplar_1'}
{'label': 2, 'time_series': [2.5, 3.6, 4.7, 5.8, 6.9], 'exemplars': [[1.5, 2.6, 3.7, 4.8, 5.9], [2.5, 3.6, 4.7, 5.8, 6.9]], 'dtw_distance_exemplar_1': 1.42828568570857, 'dtw_distance_exemplar_2': 0.0, 'closest exemplar': 'd

#### trying gini impurity reduction

In [26]:
X = [
    [1, 2, 3, 4],    # Class 0
    [1, 2, 2, 3],    # Class 0
    [10, 9, 8, 7],   # Class 1
    [11, 10, 9, 8],  # Class 1
    [2, 3, 4, 5],    # Class 0
]

# Corresponding class labels
y = [0, 0, 1, 1, 0]

In [25]:
dictionary = {}

for i in y:
    if i in dictionary:
        dictionary[i] += 1
    else:
        dictionary[i] = 1
print(dictionary)

{0: 3, 1: 2}


In [ ]:
from collections import Counter

y = [0, 0, 1, 1, 0]
dictionary = Counter(y)
print(dictionary)